In [11]:
import os
import os.path
import sys
import pandas as pd
import numpy as np
from importlib import reload

In [12]:
LOCAL_PATH = 'data'
RAW_TRACE = 'Skype_HongKong.pcapng'
TCP_TRACE_FEATURE_FILE = 'Skype_HongKong_tcp_pkt.csv'
UDP_TRACE_FEATURE_FILE = 'Skype_HongKong_udp_pkt.csv'
TCP_FLOW_FEATURE_FILE = 'Skype_HongKong_tcp_flow.csv'
UDP_FLOW_FEATURE_FILE = 'Skype_HongKong_udp_flow.csv'
# BUCKET_NAME = 'edu.nyu.hsn.ddos-data' # replace with your bucket name
# KEY = 'CAP_NIC1_00931_20130727230801.dms' # replace with your object key

In [13]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [14]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))

Conversion done
CPU times: user 37.7 s, sys: 500 ms, total: 38.2 s
Wall time: 1min 6s
Conversion done
CPU times: user 1.42 s, sys: 38.1 ms, total: 1.46 s
Wall time: 21.3 s


In [7]:
def to_csv(trace_df, extract_func, file):
    extract_func(trace_df, 1.0, upsampled=True).to_csv(file, index=False)

In [8]:
from python import flow_feature
reload(flow_feature)

tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if tcp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.tcp_generate, os.path.join(LOCAL_PATH, TCP_FLOW_FEATURE_FILE))
    elif udp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.udp_generate, os.path.join(LOCAL_PATH, UDP_FLOW_FEATURE_FILE))
    else:
        raise

/Users/shuojiang/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
UDP 1.0 samp rate with upsampling:   3%|▎         | 7/216 [00:00<00:02, 69.92it/s]

['90.217.83.107,192.168.1.163:6769', '192.168.1.163,90.217.83.107:28331', '-1']
['90.217.83.107,192.168.1.163:6769', '192.168.1.163,90.217.83.107:28331', '-1']


AssertionError: 

UDP 1.0 samp rate with upsampling:   5%|▌         | 11/216 [00:20<00:02, 69.92it/s]